In [6]:
import requests as r
import pandas as pd
import json
import logging
import logging

logging.basicConfig(level=logging.WARNING)

with open('.env-python.local') as f:
    env_vars = json.load(f)
    CLIENT_ID = env_vars["CLIENT_ID"]

URL_API = "https://api.fib.upc.edu/v2"
LANGUAGE = "en"


def make_request(endpoint):
    params = {"client_id": CLIENT_ID, "format": "json"}
    headers = {"Accept-Language": LANGUAGE}
    return r.get(f"{URL_API}/{endpoint}/", params=params, headers=headers)


In [7]:
response = make_request('assignatures')

if response.ok:
    try:
        response_json = json.loads(response.text)
        if 'results' not in response_json.keys():
            logging.error(f"results not in JSON object")
            raise RuntimeError(f"results not in JSON object")
        response_json = response_json['results']
    except Exception as e:
        logging.error(f"Error while parsing JSON object of assignatures/: {e}")
        raise RuntimeError(f"Error while parsing JSON object of assignatures/: {e}")
else:
    logging.error(f"API call to assignatures/ failed: <{response.code}>: {response.text}")
    raise RuntimeError(f"API call to assignatures/ failed: <{response.code}>: {response.text}")

In [8]:
df = pd.DataFrame(response_json)
df = df[['id', 'nom', 'credits', 'vigent']]
df = df[df['vigent'] == 'S']
df = df.drop(columns=['vigent'])
df = df.rename(columns={'id': 'id', 'nom': 'name', 'credits': 'creditHours'})
df.head()

,id,name,creditHours
0,LMD-GCED,Logic and Discrete Mathematics,7.5
1,BDA-GCED,Advanced Databases,6.0
2,AHCT-MAI,Assistive and Health-Care Technologies,4.5
3,APC,PC Architecture,6.0
4,ASDP,Academic Skills for Developing a Project,6.0


In [9]:
def get_description(id):
    response = make_request(f'assignatures/{id}/guia')

    if response.ok:
        try:
            response_json = json.loads(response.text)
            if 'descripcio' not in response_json.keys():
                raise RuntimeError(f"descripcio not in JSON object")
            return response_json['descripcio']
        except Exception as e:
            logging.warning(f"Error while parsing JSON object of assignatures/{id}/guia/: {e}")
            return None
    else:
        logging.warning(f"API call to assignatures/{id}/guia/ failed: <{response.code}>: {response.text}")
        return None
    
def make_subject_url(id):
    if LANGUAGE == "en":
        return f"https://www.fib.upc.edu/en/studies/bachelors-degrees/bachelor-degree-informatics-engineering/curriculum/syllabus/{id}"
    elif LANGUAGE == "ca":
        return f"https://www.fib.upc.edu/ca/estudis/graus/grau-en-enginyeria-informatica/pla-destudis/assignatures/{id}"
    else:
        return f"https://www.fib.upc.edu/es/estudios/grados/grado-en-ingenieria-informatica/plan-de-estudios/asignaturas/{id}"
    
df = pd.DataFrame(df.head(5))
df['description'] = df['id'].apply(get_description)
df['url'] = df['id'].apply(make_subject_url)


In [10]:
df

,id,name,creditHours,description,url
0,LMD-GCED,Logic and Discrete Mathematics,7.5,Discrete mathematics is the branch of mathemat...,https://www.fib.upc.edu/en/studies/bachelors-d...
1,BDA-GCED,Advanced Databases,6.0,This subject trains students in the skills nee...,https://www.fib.upc.edu/en/studies/bachelors-d...
2,AHCT-MAI,Assistive and Health-Care Technologies,4.5,Assistive technologies and their incidence on ...,https://www.fib.upc.edu/en/studies/bachelors-d...
3,APC,PC Architecture,6.0,In this subject the students learn how the dif...,https://www.fib.upc.edu/en/studies/bachelors-d...
4,ASDP,Academic Skills for Developing a Project,6.0,This course aims to help students to develop t...,https://www.fib.upc.edu/en/studies/bachelors-d...
